一个温度计没有单位，我们将以我们喜欢的单位建立一个读数和相应的温度值的数据集，选择一个模型，迭代地调整它的权重，直到误差的度量足够低，最终能够以我们理解的单位来解释新的读数。

In [1]:

import torch

In [68]:
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)



In [69]:
# 选择一个线性模型
def model(t_u,w,b):
    return w * t_u + b

def loss_fn(tp,tc):
    squared_diffs = (tp-tc)**2
    return squared_diffs.mean()

In [11]:
# 初始化参数
w = torch.ones(())
b = torch.zeros(())

t_p = model(t_u,w,b)
t_p

tensor([35.7000, 55.9000, 58.2000, 81.9000, 56.3000, 48.9000, 33.9000, 21.8000,
        48.4000, 60.4000, 68.4000])

In [21]:
loss = loss_fn(t_p,t_c)
loss

tensor(1763.8846)

In [56]:
delta = 0.1
loss_rate_of_change_w = (loss_fn(model(t_u,w+delta,b),t_c)-loss_fn(model(t_u,w-delta,b),t_c))/(2.0*delta)
loss_rate_of_change_b = (loss_fn(model(t_u,w,b+delta),t_c)-loss_fn(model(t_u,w,b-delta),t_c))/(2.0*delta)

In [57]:
lr=1e-2
w = w - lr * loss_rate_of_change_w
b = b - lr * loss_rate_of_change_b

In [78]:
# 改用计算梯度
def dloss_fn(t_p,t_c):
    dsq_diffs = 2*(t_p - t_c) /t_p.size(0)
    return dsq_diffs

In [79]:
def dmodel_dw(t_u,w,b):
    return t_u
def dmodel_db(t_u,w,b):
    return 1.0

In [80]:
def grad_fn(t_u,t_c,t_p,w,b):
    dloss_dtp = dloss_fn(t_p,t_c)
    dloss_dw = dloss_dtp * dmodel_dw(t_u,w,b)
    dloss_db = dloss_dtp * dmodel_db(t_u,w,b)
    return torch.stack([dloss_dw.sum(),dloss_db.sum()])

In [83]:
def training_loop(n_epochs,learning_rate,params,t_u,t_c):
    for epoch in range(1,n_epochs+1):
        w,b = params
        t_p = model(t_u,w,b)           #前向传播
        loss = loss_fn(t_p,t_c)
        grad = grad_fn(t_u,t_c,t_p,w,b)
        
        params = params - learning_rate*grad
        
        print('Epoch %d,Loss %f'%(epoch,float(loss)))
    return params

In [87]:
training_loop(n_epochs=100,learning_rate=1e-4,params=torch.tensor([1.0,0.0]),
             t_u = t_u,
             t_c = t_c)

Epoch 1,Loss 1763.884644
Epoch 2,Loss 323.090546
Epoch 3,Loss 78.929634
Epoch 4,Loss 37.552845
Epoch 5,Loss 30.540285
Epoch 6,Loss 29.351152
Epoch 7,Loss 29.148882
Epoch 8,Loss 29.113848
Epoch 9,Loss 29.107145
Epoch 10,Loss 29.105242
Epoch 11,Loss 29.104168
Epoch 12,Loss 29.103222
Epoch 13,Loss 29.102297
Epoch 14,Loss 29.101379
Epoch 15,Loss 29.100470
Epoch 16,Loss 29.099548
Epoch 17,Loss 29.098631
Epoch 18,Loss 29.097715
Epoch 19,Loss 29.096796
Epoch 20,Loss 29.095884
Epoch 21,Loss 29.094959
Epoch 22,Loss 29.094049
Epoch 23,Loss 29.093134
Epoch 24,Loss 29.092213
Epoch 25,Loss 29.091297
Epoch 26,Loss 29.090382
Epoch 27,Loss 29.089460
Epoch 28,Loss 29.088549
Epoch 29,Loss 29.087635
Epoch 30,Loss 29.086718
Epoch 31,Loss 29.085808
Epoch 32,Loss 29.084888
Epoch 33,Loss 29.083965
Epoch 34,Loss 29.083057
Epoch 35,Loss 29.082142
Epoch 36,Loss 29.081219
Epoch 37,Loss 29.080309
Epoch 38,Loss 29.079393
Epoch 39,Loss 29.078474
Epoch 40,Loss 29.077559
Epoch 41,Loss 29.076653
Epoch 42,Loss 29.07573

tensor([ 0.2327, -0.0438])

In [89]:
# 为解决lr不能同时兼顾两个参数的问题（w和b差别较大），可以改变输入，即标准化
t_un = 0.1*t_u
training_loop(n_epochs=100,learning_rate=1e-2,params=torch.tensor([1.0,0.0]),
             t_u = t_un,
             t_c = t_c)

Epoch 1,Loss 80.364342
Epoch 2,Loss 37.574917
Epoch 3,Loss 30.871077
Epoch 4,Loss 29.756193
Epoch 5,Loss 29.507149
Epoch 6,Loss 29.392458
Epoch 7,Loss 29.298828
Epoch 8,Loss 29.208717
Epoch 9,Loss 29.119417
Epoch 10,Loss 29.030487
Epoch 11,Loss 28.941875
Epoch 12,Loss 28.853565
Epoch 13,Loss 28.765556
Epoch 14,Loss 28.677851
Epoch 15,Loss 28.590431
Epoch 16,Loss 28.503321
Epoch 17,Loss 28.416496
Epoch 18,Loss 28.329975
Epoch 19,Loss 28.243738
Epoch 20,Loss 28.157801
Epoch 21,Loss 28.072151
Epoch 22,Loss 27.986799
Epoch 23,Loss 27.901731
Epoch 24,Loss 27.816954
Epoch 25,Loss 27.732460
Epoch 26,Loss 27.648256
Epoch 27,Loss 27.564342
Epoch 28,Loss 27.480711
Epoch 29,Loss 27.397358
Epoch 30,Loss 27.314295
Epoch 31,Loss 27.231512
Epoch 32,Loss 27.149006
Epoch 33,Loss 27.066790
Epoch 34,Loss 26.984844
Epoch 35,Loss 26.903173
Epoch 36,Loss 26.821791
Epoch 37,Loss 26.740675
Epoch 38,Loss 26.659838
Epoch 39,Loss 26.579279
Epoch 40,Loss 26.498987
Epoch 41,Loss 26.418974
Epoch 42,Loss 26.339228
E

tensor([ 2.7553, -2.5162])